# Study of the resulting distributions
In this notebook we want to compare the resulting distributions by applying various statistical methods to quantify differences and similarities.
## Frequencies of Linguistic Elements
To assess the similarity of the resulting distributions, we compute the [*Wasserstein-Metric*](https://en.wikipedia.org/wiki/Wasserstein_metric) of each pair of distributions.
The Wasserstein-Metric is a distance function for probability distributions. It satisfies the criteria of actual mathematical metrics and thus enables comparisons between probability distributions. Assuming one-dimensional distributions, the Wasserstein Distance of two empirical samples $P$ and $Q$ with respective random variables $X_1,...,X_n$ and $Y_1,...,Y_n$ is given by:
$$ W_{p}(P,Q)=\left({\frac {1}{n}}\sum _{i=1}^{n}\|X_{(i)}-Y_{(i)}\|^{p}\right)^{\frac {1}{p}}$$ 
Let's compute the Wasserstein-Metric for the frequencies of Characters, Words, Named Entities and Queries:


In [ ]:
from thesis_schneg.vis_modules import _get_results_paths, load_results
import numpy as np

analyses = []
# set analysis that should be analyzed
# analyses.append('query-frequencies')
analyses.append('extract-named-entities')
analyses.append('extract-words')
analyses.append('extract-chars')

# test_data = True
test_data = False

cleaned_aql = True

col = ['count()']

Load Data:

In [ ]:
from time import time
# load data
analysis_data = {}
for analysis in analyses:
    print(f"Start loading \"{analysis}\"")
    datasets = {}
    for dataset in ["aol","aql","ms-marco","orcas"]:
        start_time = time()
        paths = _get_results_paths(dataset, analysis, cleaned_aql)
        result_data = load_results(paths, test_data=test_data, cols=col)
        datasets.update({dataset: result_data})
        end_time = time()
        print(f"{dataset.upper()} loaded in {(end_time - start_time)/60} min")
    analysis_data.update({analysis: datasets})

In [ ]:
from thesis_schneg.vis_modules import get_max_x
threshs = [0.84, 0.957, 0.999]
threshs = [0.99, 0.99, 0.999]

data = analysis_data[analyses[2]]['aql'] 
print(len(data))
x_max = get_max_x(analysis_data[analyses[2]], "count()", rank_sized=True, threshold=0.999)
print(x_max)

for key,val in analysis_data[analyses[2]].items():
    print(key)
    # print(val)
    print(len(val))

    print("---------------")

Compute Wasserstein-Distances:

In [ ]:
from thesis_schneg.vis_modules import get_max_x
from numpy import ndarray
from pandas import DataFrame
from scipy.stats import wasserstein_distance_nd

# we choose thresholds so that the aqls elemntes are aligned with the highest number of elements present in the other datasets
threshs = [0.84, 0.957, 0.999]
# threshs = [0.99, 0.99, 0.999]

cnt=0
distances_data = {}
for analysis, datasets in analysis_data.items():
    print(analysis)
    x_max = get_max_x(datasets, "count()", rank_sized=True, threshold=threshs[cnt])
    print(x_max)
    # distances = ndarray((len(datasets), len(datasets)))
    distances = DataFrame(np.zeros((len(datasets), len(datasets))), index=datasets.keys(), columns=datasets.keys())
    names = []
    j=0
    for dataset_name, data in datasets.items():
        data1 = data.sort_values("count()", ascending=False)/data['count()'].sum()
        # print(data1["count()"].iloc[0:3])
        # print(data1["count()"].iloc[-3:-1])

        # data1 = data.sort_values("count()", ascending=True)
        names.append(dataset_name)
        x_vals = np.arange(1, x_max+1)
        y_vals = data1['count()'][0:x_max].to_numpy()
        # 
        if len(y_vals) < x_max:
            y_vals = np.append(y_vals, np.zeros(x_max-len(y_vals)))
        i=0
        for dataset_name, data in datasets.items():
            if dataset_name in names:
                dist = 0
            else:
                data2 = data.sort_values("count()", ascending=False)/data['count()'].sum()
                # data2 = data.sort_values("count()", ascending=True)
                x_vals2 = np.arange(1, x_max+1)
                y_vals2 = data2['count()'][0:x_max].to_numpy()
                if len(y_vals2) < x_max:
                    y_vals2 = np.append(y_vals2, np.zeros(x_max-len(y_vals2)))
                dist = wasserstein_distance_nd(x_vals, x_vals2, u_weights=y_vals, v_weights=y_vals2)
            # distances[i][j] = dist
            distances.iloc[i, j] = dist
            i+=1
        j+=1
    distances = distances + distances.T
    distances_data.update({analysis: distances})
    cnt+=1
for key, value in distances_data.items():
    print(value)

## get avarage wasserstein distances per query log
avg_distances = {}
for analysis, distances in distances_data.items():
    avg_distances.update({analysis: distances.mean().mean()})
    
for key, value in avg_distances.items():
    print(key, value)

In [ ]:
# compute average distance within AOL, MS-MARCO, ORCAS,
# compute average distance between AQL and the other datasets
# compute difference between the average distance of AQL and the average distance of the other datasets
# compute std of the distances within comparison group
# express difference in std
def get_ws_stats(distancedata):
    ws_stats = {}
    for analysis, distances in distancedata.items():
        # get distances of AOL, MS MARCO, ORCAS
        aol = distances.loc['aol'].to_numpy()
        # print(aol)
        ms_marco = distances.loc['ms-marco'].to_numpy()
        # get distances of AQL
        aql = distances.loc['aql'].to_numpy()
        # print(aol,aql,ms_marco,orcas,sep='\n')
        aql_dist = aql.sum()/3
        notaql_dist = (aol[2]+aol[3]+ms_marco[3])/3
        std = np.std([aol[2], aol[3], ms_marco[3]])
        aql_dist_deviation = notaql_dist - aql_dist
        aql_distinSTD = aql_dist_deviation/std
        ws_stats.update({analysis: [notaql_dist, aql_dist, aql_dist_deviation, std, aql_distinSTD]})
    return ws_stats
        
for key, value in get_ws_stats(distances_data).items():
    print(key, value)

Visualize results in Wasserstein Space:

In [ ]:
from sklearn.manifold import MDS
mds = MDS(n_components=2, dissimilarity='precomputed')
trans_dists = {}
for key, value in distances_data.items():
    print(key)
    trans_dists.update({key: mds.fit_transform(value)})

In [ ]:
from sklearn.manifold import MDS
from matplotlib import pyplot as plt
from pathlib import Path    
import scienceplots

def avg_dist_notAQL(x,y, point_matrtix):
    dist = 0
    for i in range(len(point_matrtix)):
        if i != 1:
            dist += np.sqrt((x - point_matrtix[i][0])**2 + (y - point_matrtix[i][1])**2)
    dist = dist/3
    # return dist
    # # get the average distance of the points within AOL, ORCAS, MS-MARCO Web Search    
    avg_dist_within = 0
    for i in [(0,2), (0,3), (2,3)]:
        avg_dist_within += np.sqrt((point_matrtix[i[0]][0] - point_matrtix[i[1]][0])**2 + (point_matrtix[i[0]][1] - point_matrtix[i[1]][1])**2)
    avg_dist_within = avg_dist_within/3

    # get the standard deviation of distances within AOL, ORCAS, MS-MARCO Web Search
    std_dist_within = 0
    for i in [(0,2), (0,3), (2,3)]:
        std_dist_within += (np.sqrt((point_matrtix[i[0]][0] - point_matrtix[i[1]][0])**2 + (point_matrtix[i[0]][1] - point_matrtix[i[1]][1])**2) - avg_dist_within)**2
    std_dist_within = np.sqrt(std_dist_within/3) 

    if dist <= avg_dist_within:
        return avg_dist_within
    elif dist > avg_dist_within and dist <= avg_dist_within + std_dist_within:
        return avg_dist_within + std_dist_within
    elif dist > avg_dist_within + std_dist_within and dist <= avg_dist_within + 2 * std_dist_within:
        return avg_dist_within + 2 * std_dist_within
    elif dist > avg_dist_within + 2 * std_dist_within and dist <= avg_dist_within + 3 * std_dist_within:
        return avg_dist_within + 3 * std_dist_within
    elif dist > avg_dist_within + 3 * std_dist_within and dist <= avg_dist_within + 4 * std_dist_within:
        return avg_dist_within + 4 * std_dist_within
    elif dist > avg_dist_within + 4 * std_dist_within and dist <= avg_dist_within + 5 * std_dist_within:
        return avg_dist_within + 5 * std_dist_within
    else: 
        return avg_dist_within + 6 * std_dist_within
        
def avg_dist_within(points):
    dist = 0
    pairs = [(0, 2), (0, 3), (2, 3)]  # Indizes der Punkte AOL, MS-MARCO, ORCAS
    for i, j in pairs:
        dist += np.sqrt((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2)
    return dist / len(pairs)
def std_within(points):
    dist = 0
    pairs = [(0, 2), (0, 3), (2, 3)]  # Indizes der Punkte AOL, MS-MARCO, ORCAS
    for i, j in pairs:
        dist += (np.sqrt((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2) - avg_dist_within(points))**2
    return np.sqrt(dist / len(pairs))

# use science style for plots from scienceplots library
plt.style.use(["science", "ieee"])
color =  ['tab:blue', 'tab:orange', 'tab:gray', 'tab:red']

save_vis = True
# save_vis = False
plot_legend = True
# plot_legend = False

# get meshsize
# mesh_size = 70
mesh_size = 500

num_plots = len(trans_dists)
cols = 3
rows = num_plots//cols
if num_plots % cols != 0:
    rows+=1

textwidth = 5.5129
aspect_ratio = 6/8
scale = 1.0
width = textwidth * scale *1.1
height = width * aspect_ratio * 0.40


fig,ax = plt.subplots(rows,cols, figsize=(width,height))


# for making the titles
keys = [key for key in trans_dists.keys()]
# Dynamically generate titles for each plot
titles = [f"{key.split('-')[-1].title()} in {key.split('-')[0].title()}s" for key in keys]
# Adjust titles for specific cases (e.g., "Entities" instead of "Entitys")
titles = ["Entities ranked", "Words ranked", "Chars ranked"]
print(titles)

positions = [[0.0,0.0], [1.0,0.0], [0.0,0.0]]


axes = ax.flatten()
for i in range(len(trans_dists)):
    ax = axes[i]
    ax.set_title(titles[i], fontsize=9)

    points = trans_dists[keys[i]]

    # get the highes and lowest x and y values of the points
    x_min = np.min(points[:,0])
    x_max = np.max(points[:,0])
    y_min = np.min(points[:,1])
    y_max = np.max(points[:,1])

    # scale the extreme values of x and y to get a good boundary for the plot
    # we take the scale of the min range
    range_scale = 0.4
    x_min = x_min - range_scale * (x_max - x_min)
    x_max = x_max + range_scale * (x_max - x_min)
    y_min = y_min - range_scale * (y_max - y_min)
    y_max = y_max + range_scale * (y_max - y_min)

    x = np.linspace(x_min, x_max, mesh_size)
    y = np.linspace(y_min, y_max, mesh_size)

    z = np.array([avg_dist_notAQL(i,j,points) for j in y for i in x])


    X, Y = np.meshgrid(x, y)
    Z = z.reshape(mesh_size, mesh_size)

    avg_dist = avg_dist_within(points)
    std_dist = std_within(points)

    ax.contourf(X.astype(float), Y.astype(float), Z.astype(float), 10, alpha=0.7, cmap = 'Blues', linewidths=0.5, linestyles='solid') #, cmap='Blues' 'OrRd'
    # Add contour lines for the average distance within AOL, MS-MARCO, and ORCAS and the standard deviation 
    contour = ax.contour(X.astype(float), Y.astype(float), Z.astype(float), levels=[avg_dist, avg_dist+std_dist, avg_dist+2*std_dist, avg_dist+3*std_dist, avg_dist+4*std_dist, avg_dist+5*std_dist], colors='black', linewidths=0.5, linestyles='dashed')

    # Label the contour lines with descriptive text
    text = ["$d_{{\mu}}$", "$d_{{\mu}} + \sigma$", "$d_{{\mu}} + 2 \sigma$"]
    for j, level in enumerate(contour.levels):
        # Find the position for the label (e.g., the midpoint of the contour line)
        idx = np.where(Z == level)
        if len(idx[0]) > 0:
            if j == 0:
                x_pos = positions[i][0] 
                y_pos = positions[i][1]
                ax.text(x_pos, y_pos, text[j] , fontsize=8, color='black')

    labels = [name.upper() for name in names]
    labels[2] = "MS-MARCO WS"
    # plot 3d scatter plot
    for j in range(len(points)):
        ax.scatter(points[j][0], points[j][1], c=color[j], label=labels[j], s=15, edgecolor='black', linewidth=0.3, alpha=0.7)

    ax.grid(True)
    ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))
    ax.xaxis.offsetText.set_x(1.1)  # Verschiebt den Exponenten horizontal
    ax.xaxis.offsetText.set_y(-0.9)  # Verschiebt den Exponenten vertikal
    
if plot_legend:
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.04), fancybox=False, ncol=len(names),edgecolor="black", frameon=True).get_frame().set_linewidth(0.5)

plt.tight_layout()
plt.subplots_adjust(top=0.9, bottom=0.1, left=0.1, right=0.9, hspace=0.4, wspace=0.3)
# plt.show()

base_path = "/home/benjamin/studium/masterarbeit/thesis-schneg/plots/"
if plot_legend:
    vis_dir = Path(f"{base_path}Wasserstein-Distances-elements-2d-contour")
else:
    vis_dir = Path(f"{base_path}Wasserstein-Distances-elements-2d-contour-no-legend")
if not vis_dir.exists() and save_vis:
        vis_dir.mkdir(parents=True)

if not vis_dir.exists() and save_vis:
        vis_dir.mkdir(parents=True)

if save_vis:
    fig.savefig(vis_dir.joinpath("all.pdf"), format='pdf')

## Length-related Frequencies
### 1. Kolmogerov-Smirnov-Test
First of all, we carry out a statistical test to check if the different distributions result from a common underlying distribution. We select the [*Kolmogerov-Smirnov-Test*](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test#Two-sample_Kolmogorov.E2.80.93Smirnov_test) to verify this hypothesis.  

The test computes a test statistic $D$ that is compared to a threshold $D_{\alpha}$ with a significance level $\alpha$. If
$$D < D_{\alpha}$$ 
then the two distributions are likely to be similar. If, in contrast, 
$$D \ge D_{\alpha}$$ 
then we can conclude the samples are not from a common distribution with a significance of $1-\alpha$. The test measures the maximum distance of the cumulative distribution functions $F$ of the involved distributions. Accordingly, $D$ is computed by
$$D = \mathrm{max} |( F_{1} - F_{2} )| $$
The threshold can be determined by 
$$D_{\alpha} = K_{\alpha}\sqrt{(n_1+n_2)/(n_1 \cdot n_2)} $$
$K_{\alpha}$ is a constant that is dependent on the significance level $\alpha$ and can be obtained by this table:


![image.png](attachment:image.png)

In [ ]:
from thesis_schneg.model import DatasetName

##### set visualization configuration #####



# select dataset (if none is selected all datasets are visualized in a joint plot) 
dataset: DatasetName = None 
# dataset: DatasetName = 'aol'
# dataset: DatasetName = 'aql'
# dataset: DatasetName = 'ms-marco'
# dataset: DatasetName = 'orcas'

# plot cleaned aol data or not
cleaned_aol = False
# cleaned_aol = True

struc_level = []
struc_level.append('queries')
struc_level.append('named-entities')
struc_level.append('words')

base_analyses = []
base_analyses.append('character-count-frequencies') #-{struc}
base_analyses.append('word-count-frequencies')
base_analyses.append('entity-count-frequencies')

analyses = []
for item in base_analyses:
    for struc in struc_level:
        if item == 'word-count-frequencies' and struc != 'words':
            analyses.append(f'{item}-{struc}')
        elif item == 'entity-count-frequencies' and struc != 'named-entities' and struc != 'words':
            analyses.append(f'{item}-{struc}')
        elif item == 'character-count-frequencies':
            analyses.append(f'{item}-{struc}')

# test_data = True
test_data = False

normalize_data = True
# normalize_data = False


In [ ]:
from thesis_schneg.vis_modules import _get_results_paths, load_results
from pathlib import Path
import time
color = None
label = None
# load data
analysis_data = []
for analysis_name in analyses:
    if dataset is None:
        result_data = {}
        # crawl files from all datasets and load into dictionary
        paths = {f"{name}": _get_results_paths(name, analysis_name, cleaned_aql=True) for name in [
            "aol", "aql", "ms-marco", "orcas"]}
        if analysis_name == 'character-count-frequencies-words' and cleaned_aol:
            base_path = Path("/mnt/ceph/storage/data-in-progress/data-teaching/theses/thesis-schneg/analysis_data/analysis/aol-words-character-count-frequencies-special")
            paths["aol"] = [path for path in base_path.iterdir(
        )]
        if analysis_name == 'character-count-frequencies-queries':
            base_path = Path("/mnt/ceph/storage/data-in-progress/data-teaching/theses/thesis-schneg/analysis_data/analysis/aql-get-char-count-special")
            paths["aql"] = [path for path in base_path.iterdir(
        )]
            
        for name, result_paths in paths.items():
            start_time = time.time()
            print(f"Loading data from {name} dataset")
            vis_data = load_results(result_paths, test_data=test_data)
            result_data.update({name: vis_data})
            end_time = time.time()  
            print(f"{name} loaded in {(end_time - start_time)/60} min")

        analysis_data.append(result_data)
    else:
        # load data from single dataset
        result_paths = _get_results_paths(dataset, analysis_name, cleaned_aql=True)
        if analysis_name == 'character-count-frequencies-words' and cleaned_aol:
            base_path = Path("/mnt/ceph/storage/data-in-progress/data-teaching/theses/thesis-schneg/analysis_data/analysis/aol-words-character-count-frequencies-special")
            result_paths = [path for path in base_path.iterdir(
        )]
        start_time = time.time()
        print(f"Loading data from {dataset} dataset")
        result_data = {dataset: load_results(result_paths)}
        end_time = time.time()  
        print(f"{dataset} loaded in {(end_time - start_time)/60} min")
        analysis_data.append(result_data)   

In [ ]:
from thesis_schneg.vis_modules import ks_test, chi2_fit

cnt = 0
for analysis in analysis_data:
    print(analyses[cnt])
    test_data = analysis['aql']
    test_counts = []
    if 'character-count' in test_data.columns:
        # print("character-count")
        test_data = test_data.query('`character-count` > 0')
        test_data = test_data.query('`character-count` < 50').sort_values('character-count', ascending=True)
        test_data = test_data['count()'].to_numpy()
        col = 'character-count'
        max = 50
    elif 'entity-count' in test_data.columns: 
        # print("entity-count")
        test_data = test_data.query('`entity-count` >= 0')
        test_data = test_data.query('`entity-count` < 3').sort_values('entity-count', ascending=True)
        test_data = test_data['count()'].to_numpy()
        col = 'entity-count'
        max = 3
    elif 'word-count' in test_data.columns:
        # print("word-count")
        test_data = test_data.query('`word-count` > 0')
        test_data = test_data.query('`word-count` < 8').sort_values('word-count', ascending=True)
        test_data = test_data['count()'].to_numpy()  
        col = 'word-count'
        max = 8
    # print(f"max: {max}")
    for dataset, data in analysis.items():
        if dataset != 'aql':
            exp_counts = []
            if col == 'entity-count':
                exp_data = data.query(f"`{col}` >= 0")
            else:
                exp_data = data.query(f"`{col}` > 0")
            exp_data = exp_data.query(f'`{col}` < {max}').sort_values(col, ascending=True)
            exp_data = exp_data['count()'].to_numpy()
            
            test_statistic, threshold, test_res = ks_test(test_data, exp_data, significance_lvl=0.05)
            print(f"Test AQL vs {dataset.upper()}: ks-test-statistic: {test_statistic} threshold: {threshold} common distribution: {test_res}")
            test_statistic, threshold, test_res = chi2_fit(test_data, exp_data, significance_lvl=0.05)
            print(f"Test AQL vs {dataset.upper()}: chi2-test-statistic: {test_statistic} threshold: {threshold} common distribution: {test_res}")

    

    cnt+=1

### 2. Wasserstein Metric
As we can see, the test clearly rejects the distributions being from the same underlying distribution. However, visualizations of the distributions showed that there are similarities. To assess, if the AQL's queries suit to the other realistic query logs, we measure distances between the distributions by computing the Wasserstein Metric.

In [ ]:
from thesis_schneg.model import DatasetName
from thesis_schneg.vis_modules import _get_results_paths, load_results
from pyarrow import compute as pc
import numpy as np

struc_level = []
struc_level.append('queries')
struc_level.append('named-Entities')
struc_level.append('words')

base_analyses = []
base_analyses.append('character-count') #-{struc}
base_analyses.append('word-count')
base_analyses.append('entity-count')

analyses = []
for item in base_analyses:
    for struc in struc_level:
        analyses.append(f'{item.lower()}-frequencies-{struc.lower()}')
    del struc_level[-1]

cleaned_aql = True

Load data:

In [ ]:
from time import time
# load data
analysis_data = {}
for analysis in analyses:
    print(f"Start loading \"{analysis}\"")
    datasets = {}
    for dataset in ["aol","aql","ms-marco","orcas"]:
        start_time = time()
        paths = _get_results_paths(dataset, analysis, cleaned_aql)
        result_data = load_results(paths)
        datasets.update({dataset: result_data})
        end_time = time()
        print(f"{dataset.upper()} loaded in {(end_time - start_time)/60} min")
    analysis_data.update({analysis: datasets})

Compute Wasserstein Distances:

In [ ]:
from thesis_schneg.vis_modules import get_max_x
from numpy import ndarray
from pandas import DataFrame
from scipy.stats import wasserstein_distance_nd

cnt=0
distances_data = {}


for analysis, datasets in analysis_data.items():
    x_max = get_max_x(datasets, f"{analysis.split('-')[0]}-count")
    distances = DataFrame(np.zeros((len(datasets), len(datasets))), index=datasets.keys(), columns=datasets.keys())
    
    names = []
    j=0
    for dataset_name, data in datasets.items():
        data1 = {dataset_name: data.sort_values(f"{analysis.split('-')[0]}-count", ascending=True)}
        names.append(dataset_name)
        x_vals = data1[dataset_name][f"{analysis.split('-')[0]}-count"][0:x_max]
        y_vals = data1[dataset_name]['count()'][0:x_max]
        i=0
        for dataset_name, data in datasets.items():
            if dataset_name in names:
                dist = 0
            else:
                data2 = data.sort_values(f"{analysis.split('-')[0]}-count", ascending=True)
                x_vals2 = data2[f"{analysis.split('-')[0]}-count"][0:x_max]
                y_vals2 = data2['count()'][0:x_max]
                dist = wasserstein_distance_nd(x_vals, x_vals2, u_weights=y_vals, v_weights=y_vals2)

            distances.iloc[i, j] = round(dist,2)

            i+=1
        j+=1
    distances = distances + distances.T
    distances_data.update({analysis: distances})
    cnt+=1
# for key, value in distances_data.items():
#     print(key)
#     print(value)

## get avarage wasserstein distances in comparison group and of aql
# avg_distances = {}
for analysis, distances in distances_data.items():
    aql_avg = distances['aql'].sum()/3
    print(f"AQLavg = {aql_avg}")
    notaql_avg = (distances['aol'][2:3].sum() + distances['ms-marco'][3].sum())/3
    print(f"notAQLavg = {notaql_avg}")

    
    # get average wasserstein distances per query log
    avg_distances = {}
    avg_distances.update({analysis: distances.mean()})
    
for key, value in avg_distances.items():
    print(key, value)

# get standard

In [ ]:
# compute average distance within AOL, MS-MARCO, ORCAS,
# compute average distance between AQL and the other datasets
# compute difference between the average distance of AQL and the average distance of the other datasets
# compute std of the distances within comparison group
# express difference in std
def get_ws_stats(distancedata):
    ws_stats = {}
    for analysis, distances in distancedata.items():
        # get distances of AOL, MS MARCO, ORCAS
        aol = distances.loc['aol'].to_numpy()
        # print(aol)
        ms_marco = distances.loc['ms-marco'].to_numpy()
        # get distances of AQL
        aql = distances.loc['aql'].to_numpy()
        # print(aol,aql,ms_marco,orcas,sep='\n')
        aql_dist = aql.sum()/3
        notaql_dist = (aol[2]+aol[3]+ms_marco[3])/3
        std = np.std([aol[2], aol[3], ms_marco[3]])
        aql_dist_deviation = notaql_dist - aql_dist
        aql_distinSTD = aql_dist_deviation/std
        ws_stats.update({analysis: [notaql_dist, aql_dist, aql_dist_deviation, std, aql_distinSTD]})
    return ws_stats
        
for key, value in get_ws_stats(distances_data).items():
    print(key, value)

Since we have 4 distances for each characteristic, we can display a configuration of points in the three-dimensional space that fully reflects the distances:

In [ ]:
from sklearn.manifold import MDS
mds = MDS(n_components=2, dissimilarity='precomputed')
trans_dists = {}
for key, value in distances_data.items():
    print(key)
    trans_dists.update({key: mds.fit_transform(value)})

In [ ]:
from sklearn.manifold import MDS
from matplotlib import pyplot as plt
from pathlib import Path    
import scienceplots

def avg_dist_notAQL(x,y, point_matrtix):
    dist = 0
    for i in range(len(point_matrtix)):
        if i != 1:
            dist += np.sqrt((x - point_matrtix[i][0])**2 + (y - point_matrtix[i][1])**2)
    dist = dist/3
    # return dist
    # # get the average distance of the points within AOL, ORCAS, MS-MARCO Web Search    
    avg_dist_within = 0
    for i in [(0,2), (0,3), (2,3)]:
        avg_dist_within += np.sqrt((point_matrtix[i[0]][0] - point_matrtix[i[1]][0])**2 + (point_matrtix[i[0]][1] - point_matrtix[i[1]][1])**2)
    avg_dist_within = avg_dist_within/3

    # get the standard deviation of distances within AOL, ORCAS, MS-MARCO Web Search
    std_dist_within = 0
    for i in [(0,2), (0,3), (2,3)]:
        std_dist_within += (np.sqrt((point_matrtix[i[0]][0] - point_matrtix[i[1]][0])**2 + (point_matrtix[i[0]][1] - point_matrtix[i[1]][1])**2) - avg_dist_within)**2
    std_dist_within = np.sqrt(std_dist_within/3) 

    if dist <= avg_dist_within:
        return avg_dist_within
    elif dist > avg_dist_within and dist <= avg_dist_within + std_dist_within:
        return avg_dist_within + std_dist_within
    elif dist > avg_dist_within + std_dist_within and dist <= avg_dist_within + 2 * std_dist_within:
        return avg_dist_within + 2 * std_dist_within
    elif dist > avg_dist_within + 2 * std_dist_within and dist <= avg_dist_within + 3 * std_dist_within:
        return avg_dist_within + 3 * std_dist_within
    elif dist > avg_dist_within + 3 * std_dist_within and dist <= avg_dist_within + 4 * std_dist_within:
        return avg_dist_within + 4 * std_dist_within
    elif dist > avg_dist_within + 4 * std_dist_within and dist <= avg_dist_within + 5 * std_dist_within:
        return avg_dist_within + 5 * std_dist_within
    else: 
        return avg_dist_within + 6 * std_dist_within
        
def avg_dist_within(points):
    dist = 0
    pairs = [(0, 2), (0, 3), (2, 3)]  # Indizes der Punkte AOL, MS-MARCO, ORCAS
    for i, j in pairs:
        dist += np.sqrt((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2)
    return dist / len(pairs)
def std_within(points):
    dist = 0
    pairs = [(0, 2), (0, 3), (2, 3)]  # Indizes der Punkte AOL, MS-MARCO, ORCAS
    for i, j in pairs:
        dist += (np.sqrt((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2) - avg_dist_within(points))**2
    return np.sqrt(dist / len(pairs))

# use science style for plots from scienceplots library
plt.style.use(["science", "ieee"])
color =  ['tab:blue', 'tab:orange', 'tab:gray', 'tab:red']

save_vis = True
# save_vis = False

# get meshsize
# mesh_size = 70
mesh_size = 500

num_plots = len(trans_dists)
cols = 3
rows = num_plots//cols
if num_plots % cols != 0:
    rows+=1

textwidth = 5.5129
aspect_ratio = 6/8
scale = 1.0
width = textwidth * scale
height = width * aspect_ratio * 0.85


fig,ax = plt.subplots(rows,cols, figsize=(width,height))


# for making the titles
keys = [key for key in trans_dists.keys()]

titles = ["Characters per Query", "Characters per Entity", "Characters per Word", "Words per Query", "Words per Entity", "Entities per Query"]

positions = [[0.0,-1.0], [0.0,-0.6], [0.0,0.0], [-0.4,0.0], [0.04,0.03], [0.0,0.0]]


axes = ax.flatten()
for i in range(len(trans_dists)):
    ax = axes[i]
    ax.set_title(titles[i], fontsize=8)

    points = trans_dists[keys[i]]

    # get the highes and lowest x and y values of the points
    x_min = np.min(points[:,0])
    x_max = np.max(points[:,0])
    y_min = np.min(points[:,1])
    y_max = np.max(points[:,1])

    # scale the extreme values of x and y to get a good boundary for the plot
    # we take the scale of the min range
    range_scale = 0.4
    x_min = x_min - range_scale * (x_max - x_min)
    x_max = x_max + range_scale * (x_max - x_min)
    y_min = y_min - range_scale * (y_max - y_min)
    y_max = y_max + range_scale * (y_max - y_min)

    x = np.linspace(x_min, x_max, mesh_size)
    y = np.linspace(y_min, y_max, mesh_size)

    z = np.array([avg_dist_notAQL(i,j,points) for j in y for i in x])


    X, Y = np.meshgrid(x, y)
    Z = z.reshape(mesh_size, mesh_size)

    avg_dist = avg_dist_within(points)
    std_dist = std_within(points)

    ax.contourf(X.astype(float), Y.astype(float), Z.astype(float), 10, alpha=0.7, cmap = 'Blues', linewidths=0.5, linestyles='solid') #, cmap='Blues' 'OrRd'
    # Add contour lines for the average distance within AOL, MS-MARCO, and ORCAS and the standard deviation 
    contour = ax.contour(X.astype(float), Y.astype(float), Z.astype(float), levels=[avg_dist, avg_dist+std_dist, avg_dist+2*std_dist, avg_dist+3*std_dist, avg_dist+4*std_dist, avg_dist+5*std_dist], colors='black', linewidths=0.5, linestyles='dashed')

    # Label the contour lines with descriptive text
    text = ["$d_{{\mu}}$", "$d_{{\mu}} + \sigma$", "$d_{{\mu}} + 2 \sigma$"]
    for j, level in enumerate(contour.levels):
        # Find the position for the label (e.g., the midpoint of the contour line)
        idx = np.where(Z == level)
        if len(idx[0]) > 0:
            if j == 0:
                if titles[i] == "Characters per Entity":
                    x_pos = positions[i][0] 
                    y_pos = positions[i][1]-0.5
                elif titles[i] == "Words per Query":
                    x_pos = positions[i][0] +0.5
                    y_pos = positions[i][1]
                elif titles[i] == "Words per Entity":
                    x_pos = positions[i][0] 
                    y_pos = positions[i][1] -0.1
                else:
                    x_pos = positions[i][0] 
                    y_pos = positions[i][1]
                ax.text(x_pos, y_pos, text[j] , fontsize=8, color='black')

    labels = [name.upper() for name in names]
    labels[2] = "MS-MARCO WS"
    # plot 3d scatter plot
    for j in range(len(points)):
        ax.scatter(points[j][0], points[j][1], c=color[j], label=labels[j], s=15, edgecolor='black', linewidth=0.3)

    ax.grid(True)
    

handles, labels = ax.get_legend_handles_labels()

fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.03), fancybox=False, ncol=len(names),edgecolor="black", frameon=True).get_frame().set_linewidth(0.5)

plt.tight_layout()
plt.subplots_adjust(top=0.9, bottom=0.1, left=0.1, right=0.9, hspace=0.33, wspace=0.3)
plt.show()

base_path = "/home/benjamin/studium/masterarbeit/thesis-schneg/plots/"

vis_dir = Path(f"{base_path}Wasserstein-Distances-lengths-2d-contour")
if not vis_dir.exists() and save_vis:
        vis_dir.mkdir(parents=True)

if not vis_dir.exists() and save_vis:
        vis_dir.mkdir(parents=True)

if save_vis:
    fig.savefig(vis_dir.joinpath("all.pdf"), format='pdf')

In [ ]:
for key,val in trans_dists.items():
    print(key)
    print(val)
